## Exercise 1

**Task:** *Train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?*

In [2]:
import random
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn.functional as F
from tqdm import notebook

RANDOM_SEED = 42

%matplotlib inline
sns.set_theme()

In [10]:
words = open('names.txt', 'r').read().splitlines()
words[:10] #slicing the first 10 indexes of the list

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [27]:
print(f"Number of names: {len(words):,}")

Number of names: 32,033


In [28]:
# Retrieving a set of unique letters
chars = sorted(list(set(''.join(words))))

# Creating a mapping from a letter to an id
char2id = {s: i+1 for i, s in enumerate(chars)}
# Adding the start_of_word/end_of_word token => "."
char2id['.'] = 0

# Creating a mapping from an id to letter
id2char = {i: s for s, i in char2id.items()}

In [29]:
char2id

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '.': 0}

In [30]:
id2char

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

## Biagram Model

In [32]:
# Creating the dataset

for word in words:
    chars = ['.'] + list(word) + ['.']
    for ch1, ch2

['.', 'e', 'm', 'm', 'a', '.']
['.', 'o', 'l', 'i', 'v', 'i', 'a', '.']
['.', 'a', 'v', 'a', '.']
['.', 'i', 's', 'a', 'b', 'e', 'l', 'l', 'a', '.']
['.', 's', 'o', 'p', 'h', 'i', 'a', '.']
['.', 'c', 'h', 'a', 'r', 'l', 'o', 't', 't', 'e', '.']
['.', 'm', 'i', 'a', '.']
['.', 'a', 'm', 'e', 'l', 'i', 'a', '.']
['.', 'h', 'a', 'r', 'p', 'e', 'r', '.']
['.', 'e', 'v', 'e', 'l', 'y', 'n', '.']
['.', 'a', 'b', 'i', 'g', 'a', 'i', 'l', '.']
['.', 'e', 'm', 'i', 'l', 'y', '.']
['.', 'e', 'l', 'i', 'z', 'a', 'b', 'e', 't', 'h', '.']
['.', 'm', 'i', 'l', 'a', '.']
['.', 'e', 'l', 'l', 'a', '.']
['.', 'a', 'v', 'e', 'r', 'y', '.']
['.', 's', 'o', 'f', 'i', 'a', '.']
['.', 'c', 'a', 'm', 'i', 'l', 'a', '.']
['.', 'a', 'r', 'i', 'a', '.']
['.', 's', 'c', 'a', 'r', 'l', 'e', 't', 't', '.']
['.', 'v', 'i', 'c', 't', 'o', 'r', 'i', 'a', '.']
['.', 'm', 'a', 'd', 'i', 's', 'o', 'n', '.']
['.', 'l', 'u', 'n', 'a', '.']
['.', 'g', 'r', 'a', 'c', 'e', '.']
['.', 'c', 'h', 'l', 'o', 'e', '.']
['.', 'p',

['.', 'k', 'a', 'l', 'y', 'n', '.']
['.', 'k', 'a', 'm', 'e', 'r', 'y', 'n', '.']
['.', 'k', 'a', 'o', 'i', 'r', '.']
['.', 'k', 'a', 's', 'h', 'v', 'i', '.']
['.', 'k', 'e', 'y', 'a', '.']
['.', 'k', 'e', 'y', 'a', 'n', 'a', '.']
['.', 'k', 'h', 'y', 'l', 'i', 'e', '.']
['.', 'k', 'y', 'l', 'e', 'i', '.']
['.', 'l', 'e', 'n', 'n', 'i', 'x', '.']
['.', 'l', 'o', 'g', 'a', 'n', 'n', '.']
['.', 'l', 'u', 'j', 'a', 'i', 'n', '.']
['.', 'l', 'u', 'm', 'i', '.']
['.', 'm', 'a', 'g', 'a', 'l', 'i', '.']
['.', 'm', 'a', 'k', 'a', 'e', 'l', 'a', '.']
['.', 'm', 'a', 'k', 'a', 'i', 'l', 'a', '.']
['.', 'm', 'a', 'l', 'e', 'y', 'a', 'h', '.']
['.', 'm', 'a', 'z', 'z', 'y', '.']
['.', 'm', 'e', 'l', 'a', 'h', '.']
['.', 'm', 'e', 'r', 'y', 'e', 'm', '.']
['.', 'm', 'i', 'r', 'a', 'l', '.']
['.', 'm', 'i', 't', 'h', 'r', 'a', '.']
['.', 'n', 'i', 'h', 'i', 'r', 'a', '.']
['.', 'n', 'u', 'h', 'a', '.']
['.', 'o', 'r', 'i', 'o', 'n', '.']
['.', 'q', 'u', 'e', 't', 'z', 'a', 'l', 'y', '.']
['.', 'r',

['.', 'j', 'a', 'z', 'a', 'r', 'i', 'a', '.']
['.', 'j', 'e', 'n', 'i', 'y', 'a', 'h', '.']
['.', 'j', 'e', 'r', 'i', '.']
['.', 'j', 'e', 's', 'e', 'n', 'i', 'a', '.']
['.', 'j', 'o', 'c', 'e', 'l', 'i', 'n', '.']
['.', 'j', 'o', 'r', 'd', 'y', '.']
['.', 'j', 'o', 's', 'i', 'a', 'h', '.']
['.', 'j', 'o', 'z', 'l', 'y', 'n', 'n', '.']
['.', 'j', 'r', 'u', '.']
['.', 'j', 'u', 'd', 'e', 'a', '.']
['.', 'j', 'u', 'l', 'i', 'e', 'a', 'n', 'n', '.']
['.', 'k', 'a', 'h', 'e', 'a', 'l', 'a', 'n', 'i', '.']
['.', 'k', 'a', 'l', 'l', 'y', '.']
['.', 'k', 'a', 'n', 'd', 'a', 'c', 'e', '.']
['.', 'k', 'a', 'r', 'i', 's', 'h', 'm', 'a', '.']
['.', 'k', 'a', 't', 'a', 'l', 'e', 'i', 'a', '.']
['.', 'k', 'a', 't', 'h', 'a', 'r', 'i', 'n', 'a', '.']
['.', 'k', 'a', 't', 'h', 'i', 'a', '.']
['.', 'k', 'a', 'y', 'a', 'n', '.']
['.', 'k', 'a', 'y', 's', 'l', 'e', 'e', '.']
['.', 'k', 'e', 'a', 'l', 'a', '.']
['.', 'k', 'e', 'e', 'l', 'i', 'e', '.']
['.', 'k', 'e', 'i', 'k', 'o', '.']
['.', 'k', 'e', '

['.', 'r', 'a', 'l', 'e', 'y', '.']
['.', 'r', 'a', 'm', 'a', 't', 'o', 'u', 'l', 'a', 'y', 'e', '.']
['.', 'r', 'a', 'n', 'y', 'i', 'a', 'h', '.']
['.', 'r', 'a', 't', 'e', 'e', 'l', '.']
['.', 'r', 'a', 'y', 'c', 'h', 'e', 'l', '.']
['.', 'r', 'a', 'y', 'l', 'e', 'a', '.']
['.', 'r', 'a', 'y', 'n', 'i', 'e', '.']
['.', 'r', 'a', 'z', 'i', 'a', 'h', '.']
['.', 'r', 'e', 'j', 'o', 'i', 'c', 'e', '.']
['.', 'r', 'e', 'n', 'e', 's', 'm', 'a', 'y', '.']
['.', 'r', 'e', 'n', 'i', 'y', 'a', '.']
['.', 'r', 'e', 'y', 'l', 'y', 'n', '.']
['.', 'r', 'h', 'a', 'e', 'l', 'y', 'n', '.']
['.', 'r', 'h', 'a', 'e', 'l', 'y', 'n', 'n', '.']
['.', 'r', 'h', 'i', 'y', 'a', '.']
['.', 'r', 'i', 'm', '.']
['.', 'r', 'i', 's', 'h', 'i', 'k', 'a', '.']
['.', 'r', 'o', 'n', 'i', 'y', 'a', 'h', '.']
['.', 'r', 'o', 'o', 'h', 'i', '.']
['.', 'r', 'o', 's', 'a', 'l', 'e', 'a', 'h', '.']
['.', 'r', 'o', 's', 'a', 'l', 'e', 'e', 'n', 'a', '.']
['.', 'r', 'o', 's', 'a', 'l', 'e', 'n', 'e', '.']
['.', 'r', 'o', 's

['.', 'a', 'm', 'e', 'i', 'r', 'a', 'h', '.']
['.', 'a', 'm', 'e', 'y', 'a', 'l', 'i', '.']
['.', 'a', 'm', 'e', 'y', 'a', 'l', 'l', 'i', '.']
['.', 'a', 'm', 'i', 'e', 'e', '.']
['.', 'a', 'm', 'i', 'e', 'l', 'l', 'e', '.']
['.', 'a', 'm', 'i', 'l', 'i', 'a', 'h', '.']
['.', 'a', 'm', 'i', 'l', 'l', 'i', 'a', 'n', 'n', 'a', '.']
['.', 'a', 'm', 'i', 't', 'a', '.']
['.', 'a', 'm', 'm', 'a', 'r', 'i', 'e', '.']
['.', 'a', 'm', 'o', 'g', 'h', 'a', '.']
['.', 'a', 'm', 'o', 'y', 'a', '.']
['.', 'a', 'm', 'y', 'r', 'i', 'a', '.']
['.', 'a', 'm', 'y', 'r', 'i', 'e', '.']
['.', 'a', 'n', 'a', 'h', 'i', 'a', '.']
['.', 'a', 'n', 'a', 'i', 's', 'a', 'b', 'e', 'l', '.']
['.', 'a', 'n', 'a', 'r', 'e', 'l', 'y', '.']
['.', 'a', 'n', 'a', 'r', 'i', 'a', '.']
['.', 'a', 'n', 'a', 's', 't', 'a', 'z', 'j', 'a', '.']
['.', 'a', 'n', 'a', 't', 'a', 'l', 'i', 'a', '.']
['.', 'a', 'n', 'a', 'y', 'a', 'l', 'e', 'e', '.']
['.', 'a', 'n', 'a', 'y', 'a', 't', '.']
['.', 'a', 'n', 'a', 'y', 'i', 'a', 'h', '.'

['.', 'm', 'a', 'r', 'j', 'a', 'e', '.']
['.', 'm', 'a', 'r', 'j', 'a', 'n', '.']
['.', 'm', 'a', 'r', 'k', 'e', 'l', 'l', 'a', '.']
['.', 'm', 'a', 'r', 'k', 'i', 'a', '.']
['.', 'm', 'a', 'r', 'k', 'y', 'l', 'a', '.']
['.', 'm', 'a', 'r', 'l', 'e', 'a', 'h', '.']
['.', 'm', 'a', 'r', 'l', 'e', 'y', 'a', 'h', '.']
['.', 'm', 'a', 'r', 'l', 'i', 'a', 'n', 'a', '.']
['.', 'm', 'a', 'r', 'l', 'i', 'z', 'e', '.']
['.', 'm', 'a', 'r', 'l', 'y', 's', '.']
['.', 'm', 'a', 'r', 'r', 'i', 'a', 'n', 'n', 'a', '.']
['.', 'm', 'a', 'r', 'r', 'o', 'n', '.']
['.', 'm', 'a', 'r', 'v', 'a', '.']
['.', 'm', 'a', 'r', 'y', 'b', 'e', 'l', '.']
['.', 'm', 'a', 'r', 'y', 'e', 'm', '.']
['.', 'm', 'a', 'r', 'y', 'l', 'y', 'n', '.']
['.', 'm', 'a', 'r', 'y', 'n', 'n', '.']
['.', 'm', 'a', 'r', 'y', 'o', 'r', 'i', '.']
['.', 'm', 'a', 'r', 'y', 'r', 'u', 't', 'h', '.']
['.', 'm', 'a', 's', 't', 'a', 'n', 'i', '.']
['.', 'm', 'a', 's', 'u', 'm', 'a', '.']
['.', 'm', 'a', 't', 'e', 'l', 'y', 'n', '.']
['.', 'm

['.', 'z', 'a', 'k', 'a', 'r', 'y', '.']
['.', 'z', 'a', 'y', 'l', 'i', 'n', '.']
['.', 'z', 'i', 'd', 'a', 'n', '.']
['.', 'a', 'h', 'a', 'd', '.']
['.', 'a', 'l', 'i', 'm', '.']
['.', 'a', 'r', 'j', 'a', 'n', '.']
['.', 'a', 'r', 'l', 'a', 'n', '.']
['.', 'b', 'o', 'y', '.']
['.', 'c', 'h', 'a', 'd', 'w', 'i', 'c', 'k', '.']
['.', 'c', 'o', 'd', 'a', '.']
['.', 'c', 'o', 'n', 'a', 'l', 'l', '.']
['.', 'c', 'o', 'r', 'n', 'e', 'l', 'l', '.']
['.', 'c', 'r', 'i', 's', 't', 'h', 'i', 'a', 'n', '.']
['.', 'd', 'a', 's', 'h', 'e', 'l', '.']
['.', 'd', 'a', 'v', 'e', 'n', '.']
['.', 'd', 'e', 'q', 'u', 'a', 'n', '.']
['.', 'd', 'i', 'v', 'i', 'n', 'e', '.']
['.', 'e', 'm', 'r', 'a', 'n', '.']
['.', 'e', 'r', 'i', 'n', '.']
['.', 'e', 's', 'a', 'u', '.']
['.', 'f', 'e', 'r', 'm', 'i', 'n', '.']
['.', 'h', 'a', 'd', 'l', 'e', 'y', '.']
['.', 'i', 'v', 'o', 'r', 'y', '.']
['.', 'j', 'a', 'e', 'd', 'y', 'n', '.']
['.', 'j', 'a', 'e', 'l', 'y', 'n', '.']
['.', 'j', 'a', 'h', 'd', 'i', 'e', 'l',

['.', 'b', 'r', 'a', 'v', 'e', 'r', 'y', '.']
['.', 'b', 'r', 'a', 'y', 't', 'e', 'n', '.']
['.', 'b', 'r', 'a', 'y', 'v', 'e', 'n', '.']
['.', 'b', 'r', 'e', 'l', 'a', 'n', 'd', '.']
['.', 'b', 'r', 'e', 'x', 'l', 'e', 'y', '.']
['.', 'b', 'r', 'e', 'y', 'e', 'r', '.']
['.', 'b', 'r', 'i', 'e', 'n', '.']
['.', 'b', 'r', 'i', 'g', 'h', 't', 'e', 'n', '.']
['.', 'b', 'r', 'y', 'l', 'e', 'e', '.']
['.', 'b', 'r', 'y', 'o', 'r', '.']
['.', 'c', 'a', 'e', 'l', 'e', 'b', '.']
['.', 'c', 'a', 'i', 's', 'y', 'n', '.']
['.', 'c', 'a', 'l', 'e', 'l', '.']
['.', 'c', 'a', 'l', 'e', 'm', '.']
['.', 'c', 'a', 'n', 'd', 'o', 'n', '.']
['.', 'c', 'a', 'n', 'e', 'l', 'o', '.']
['.', 'c', 'a', 'r', 'd', 'e', 'r', '.']
['.', 'c', 'a', 'r', 'd', 'i', 'e', 'r', '.']
['.', 'c', 'a', 'r', 'e', 'e', 'm', '.']
['.', 'c', 'a', 'r', 'l', 's', 'o', 'n', '.']
['.', 'c', 'a', 'r', 'w', 'y', 'n', '.']
['.', 'c', 'a', 's', '.']
['.', 'c', 'a', 's', 't', 'i', 'n', '.']
['.', 'c', 'a', 'y', 'c', 'e', '.']
['.', 'c', 

['.', 'h', 'e', 'n', 'n', 'i', 'n', 'g', '.']
['.', 'h', 'e', 'r', 'm', 'i', 'n', 'i', 'o', '.']
['.', 'h', 'i', 'l', 'l', '.']
['.', 'h', 'i', 'y', 'a', 'n', '.']
['.', 'h', 'r', 'i', 'h', 'a', 'a', 'n', '.']
['.', 'h', 'u', 'x', 's', 'o', 'n', '.']
['.', 'h', 'y', 'k', 'e', 'e', 'm', '.']
['.', 'i', 'h', 's', 'a', 'a', 'n', '.']
['.', 'i', 'm', 'e', 'r', '.']
['.', 'i', 'm', 'i', 'r', '.']
['.', 'i', 'r', 'e', 'm', 'i', 'd', 'e', '.']
['.', 'i', 's', 'a', 'h', 'i', '.']
['.', 'i', 's', 'a', 'i', 'h', '.']
['.', 'i', 's', 'h', 'a', 'k', '.']
['.', 'i', 's', 'h', 'a', 'n', 'k', '.']
['.', 'i', 's', 's', 'a', 'c', 'h', 'a', 'r', '.']
['.', 'i', 's', 's', 'a', 'm', '.']
['.', 'i', 's', 's', 'i', 'a', 'h', '.']
['.', 'i', 'y', 'a', 'a', 'n', '.']
['.', 'i', 'z', 'a', 'i', '.']
['.', 'i', 'z', 'i', 'c', 'k', '.']
['.', 'j', 'a', 'b', 'b', 'a', 'r', '.']
['.', 'j', 'a', 'b', 'r', 'i', '.']
['.', 'j', 'a', 'c', 'c', 'o', 'b', '.']
['.', 'j', 'a', 'd', 'a', 'r', 'r', 'i', 'u', 's', '.']
['.',

['.', 'a', 'h', 'm', 'a', 'n', '.']
['.', 'a', 'h', 'm', 'e', 'i', 'r', '.']
['.', 'a', 'h', 's', 'a', 'a', 'd', '.']
['.', 'a', 'h', 's', 'i', 'r', '.']
['.', 'a', 'i', 'i', 'd', 'e', 'n', '.']
['.', 'a', 'i', 'r', 'o', 'n', '.']
['.', 'a', 'i', 's', 'a', 'i', 'a', 'h', '.']
['.', 'a', 'i', 's', 'e', 'a', '.']
['.', 'a', 'i', 's', 'e', 'n', '.']
['.', 'a', 'i', 't', 'o', '.']
['.', 'a', 'i', 'v', 'a', 'n', '.']
['.', 'a', 'i', 'y', 'a', 'n', '.']
['.', 'a', 'j', 'a', 'v', 'i', 'o', 'n', '.']
['.', 'a', 'j', 'i', 't', '.']
['.', 'a', 'k', 'h', 'a', 'i', '.']
['.', 'a', 'k', 'h', 'i', 'l', 'l', 'e', 's', '.']
['.', 'a', 'k', 'i', 'e', 'l', '.']
['.', 'a', 'k', 'i', 'e', 'm', '.']
['.', 'a', 'k', 'o', 'i', '.']
['.', 'a', 'k', 'o', 'r', 'i', '.']
['.', 'a', 'k', 's', 'h', 'a', 't', '.']
['.', 'a', 'k', 's', 'h', 'i', 't', 'h', '.']
['.', 'a', 'l', 'a', 'n', 'd', '.']
['.', 'a', 'l', 'a', 'n', 'd', 'i', 's', '.']
['.', 'a', 'l', 'a', 'n', 'i', '.']
['.', 'a', 'l', 'a', 's', 't', 'a', 'r',